## Imports & Data download

In [17]:
%%capture --no-stderr --no-display
# NBVAL_IGNORE_OUTPUT

try:
  import secml
except ImportError:
  %pip install git+https://github.com/pralab/secml

In [19]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, roc_curve, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import VotingClassifier, RandomForestClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from lightgbm import LGBMClassifier
import pickle
import requests
import os


In [20]:
goodware_train_url = 'https://raw.githubusercontent.com/SamuReyes/MAADM-SEGUR/main/dataset/2000_goodware_2018_2020_features.csv'
malware_train_url = 'https://raw.githubusercontent.com/SamuReyes/MAADM-SEGUR/main/dataset/2000_malware_2018_2020_features.csv'
goodware_test_url = 'https://raw.githubusercontent.com/SamuReyes/MAADM-SEGUR/main/dataset/goodware_features.csv'
malware_test_url = 'https://raw.githubusercontent.com/SamuReyes/MAADM-SEGUR/main/dataset/malware_features.csv'

folder = 'dataset'
os.makedirs(folder, exist_ok=True)
goodware_train_filename = os.path.join(folder, '2000_goodware_2018_2020_features.csv')
malware_train_filename = os.path.join(folder, '2000_malware_2018_2020_features.csv')
goodware_test_filename = os.path.join(folder, 'goodware_features.csv')
malware_test_filename = os.path.join(folder, 'malware_features.csv')

# Descarga los archivos
for url, local_filename in zip([goodware_train_url, malware_train_url, goodware_test_url, malware_test_url],
                                [goodware_train_filename, malware_train_filename, goodware_test_filename, malware_test_filename]):
    response = requests.get(url)
    response.raise_for_status()
    with open(local_filename, 'wb') as f:
        f.write(response.content)

## Data Preparation

In [21]:
goodware = pd.read_csv('./dataset/2000_goodware_2018_2020_features.csv')
malware = pd.read_csv('./dataset/2000_malware_2018_2020_features.csv')
goodware_val = pd.read_csv('./dataset/goodware_features.csv')
malware_val = pd.read_csv('./dataset/malware_features.csv')

In [22]:
def prepare_data(goodware, malware):
    # Asignar etiquetas
    goodware['label'] = 0
    malware['label'] = 1

    # Concatenar y mezclar los datos
    data = pd.concat([goodware, malware], ignore_index=True)
    data = data.sample(frac=1).reset_index(drop=True)

    # Calcular diferencia de versiones y limpiar datos
    data['diff_andrversion'] = data['target_andrversion'] - data['min_andrversion']
    data = data.drop(['hash', 'PackageName', 'max_andrversion', 'target_andrversion', 'min_andrversion'], axis=1)

    return data

# Preparar los conjuntos de datos de entrenamiento y validación
data = prepare_data(goodware, malware)
data_val = prepare_data(goodware_val, malware_val)
data

,android.permission.ACCESS_ALL_DOWNLOADS,android.permission.ACCESS_BLUETOOTH_SHARE,android.permission.ACCESS_CACHE_FILESYSTEM,android.permission.ACCESS_CHECKIN_PROPERTIES,android.permission.ACCESS_CONTENT_PROVIDERS_EXTERNALLY,android.permission.ACCESS_DOWNLOAD_MANAGER,android.permission.ACCESS_DOWNLOAD_MANAGER_ADVANCED,android.permission.ACCESS_DRM_CERTIFICATES,android.permission.ACCESS_EPHEMERAL_APPS,android.permission.ACCESS_FM_RADIO,...,EditViewCount,ImageButtonCount,CheckBoxCount,RadioGroupCount,RadioButtonCount,ToastCount,SpinnerCount,ListViewCount,label,diff_andrversion
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,7
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,8
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,12
3,0,0,0,0,0,0,0,0,0,0,...,2,3,1,0,0,0,0,19,0,10
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,8
3996,0,0,0,0,0,0,0,0,0,0,...,3,3,4,2,3,0,45,38,0,9
3997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,12
3998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,13


In [35]:
# Separar las características y etiquetas
X, y = data.drop('label', axis=1), data['label']
X_val, y_val = data_val.drop('label', axis=1), data_val['label']

# Dividir los datos de entrenamiento para crear un subconjunto de validación
X_train, X_val2, y_train, y_val2 = train_test_split(X, y, test_size=0.1)

# Combinar los conjuntos de validación
X_val = pd.concat([X_val, X_val2], ignore_index=True)
y_val = pd.concat([y_val, y_val2], ignore_index=True)


## Classification

Durante el desarrollo y las pruebas de los adversarial attacks, en particular los métodos basados en gradientes, como Projected Gradient Descent Least Squares (PGD-LS), fue necesario cambiar la selección de modelos. Inicialmente, se emplearon clasificadores tradicionales, como los disponibles en scikit-learn o catboost. Sin embargo, estos modelos carecen de la capacidad de proporcionar información de gradiente con respecto a sus entradas. Esta es una característica crucial necesaria para la implementación efectiva de ataques adversariales basados en gradientes, que dependen en gran medida de esta información de gradiente para modificar las entradas de forma que se maximice el error de predicción.

En consecuencia, para dar cabida a los requisitos de PGD-LS y ataques similares, el modelo clasificador se transformó en una red neuronal (NN). Las redes neuronales, por su diseño, admiten intrínsecamente el cálculo de gradientes debido a su naturaleza diferenciable. Este cambio nos permitió el acceso directo a los gradientes, esenciales para calcular la dirección y magnitud de las perturbaciones de entrada necesarias para engañar al modelo.

### Random Forest (does not support gradient-based attacks like PGD-LS)

In [ ]:
# from secml.ml.classifiers.sklearn.c_classifier_random_forest import CClassifierRandomForest
# from secml.data import CDataset
# from secml.ml.features.normalization import CNormalizerMinMax
# from sklearn.model_selection import train_test_split

# # Initialize the RandomForest classifier from secml
# clf = CClassifierRandomForest(
#     n_estimators=50,
#     criterion='gini',
#     max_depth=10,
#     min_samples_split=2,
#     random_state=42
# )

# X_train, X_val, y_train, y_val = train_test_split(data.drop('label', axis=1), data['label'], test_size=0.1, random_state=42)

# # Convert to CDataset
# tr = CDataset(X_train.values, y_train.values)
# ts = CDataset(X_val.values, y_val.values)

# # Normalize the data
# nmz = CNormalizerMinMax()
# tr.X = nmz.fit_transform(tr.X)
# ts.X = nmz.transform(ts.X)

# # Training the RandomForest classifier
# clf.fit(tr.X, tr.Y)

CClassifierRandomForest{'sklearn_model': RandomForestClassifier(max_depth=10, n_estimators=50, n_jobs=1, random_state=42), 'classes': CArray(2,)(dense: [0 1]), 'n_features': 359, 'preprocess': None}

### NN (binary output)

In [24]:
from secml.data import CDataset
from secml.ml.features.normalization import CNormalizerMinMax
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(data.drop('label', axis=1), data['label'], test_size=0.1, random_state=42)

# Convert to CDataset
tr = CDataset(X_train.values, y_train.values)
ts = CDataset(X_val.values, y_val.values)

# Normalize the data
nmz = CNormalizerMinMax()
tr.X = nmz.fit_transform(tr.X)
ts.X = nmz.transform(ts.X)

#### Simple NN

In [25]:
import torch
from torch import nn
import torch.optim as optim
from secml.array import CArray
from secml.ml.classifiers import CClassifierPyTorch

class SimpleNN(nn.Module):
    def __init__(self, num_features):
        super(SimpleNN, self).__init__()

        # Binary classification, output should be 2
        self.fc = nn.Linear(num_features, 2)

    def forward(self, x):
        return self.fc(x)


# Reinstantiate and retrain if necessary
num_features = tr.X.shape[1]  # Number of input features
nn_model = SimpleNN(num_features)
optimizer = optim.SGD(nn_model.parameters(), lr=0.01)
loss_function = nn.CrossEntropyLoss()

clf = CClassifierPyTorch(
    model=nn_model,
    loss=loss_function,
    optimizer=optimizer,
    epochs=30,
    batch_size=512,
    input_shape=(num_features,)
)

# Convert training data to CArray if not already done
tr_X_ca = CArray(tr.X)
tr_Y_ca = CArray(tr.Y, dtype=int)  # Ensure labels are integer type

# Fit the model again
clf.fit(tr_X_ca, tr_Y_ca)

# After retraining, check the shapes again
logits_train = clf.predict(tr_X_ca).tondarray()
logits_val = clf.predict(ts.X).tondarray()
print("New Shape of logits_train:", logits_train.shape)
print("New Shape of logits_val:", logits_val.shape)

New Shape of logits_train: (3600,)
New Shape of logits_val: (400,)


#### Enhanced NN (Solo de pruebas, utilizar el primer NN es suficiente)

In [ ]:
# import torch
# from torch import nn
# import numpy as np
# import torch.optim as optim
# from secml.array import CArray
# from secml.ml.classifiers import CClassifierPyTorch

# class EnhancedNN(nn.Module):
#     def __init__(self, num_features):
#         super(EnhancedNN, self).__init__()

#         # More complex structure
#         self.layer1 = nn.Linear(num_features, 128)  # First hidden layer
#         self.relu1 = nn.ReLU()  # Activation function for first layer
#         self.dropout1 = nn.Dropout(0.5)  # Dropout for regularization
#         self.layer2 = nn.Linear(128, 64)  # Second hidden layer
#         self.relu2 = nn.ReLU()  # Activation function for second layer
#         self.dropout2 = nn.Dropout(0.5)  # Another dropout layer
#         self.output_layer = nn.Linear(64, 2)  # Output layer for binary classification

#     def forward(self, x):
#         x = self.layer1(x)
#         x = self.relu1(x)
#         x = self.dropout1(x)
#         x = self.layer2(x)
#         x = self.relu2(x)
#         x = self.dropout2(x)
#         x = self.output_layer(x)
#         return x

# # Reinstantiate the model with enhanced architecture
# num_features = tr.X.shape[1]
# enhanced_model = EnhancedNN(num_features)
# optimizer = optim.SGD(enhanced_model.parameters(), lr=0.01)
# loss_function = nn.CrossEntropyLoss()

# clf = CClassifierPyTorch(
#     model=enhanced_model,
#     loss=loss_function,
#     optimizer=optimizer,
#     epochs=15,
#     batch_size=256,
#     input_shape=(num_features,)
# )

# # Fit the model
# tr_X_ca = CArray(tr.X)
# tr_Y_ca = CArray(tr.Y, dtype=int)
# clf.fit(tr_X_ca, tr_Y_ca)

### Metrics

In [26]:
import torch
import numpy as np
from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt
import seaborn as sns

# Function to convert logits to probabilities using softmax
def softmax(logits):
    e_x = np.exp(logits - np.max(logits, axis=1, keepdims=True))
    return e_x / e_x.sum(axis=1, keepdims=True)

# Use the model directly for predictions for metrics calculations
test_tensor_train = torch.FloatTensor(tr.X.tondarray())  # Convert training data to tensors
test_tensor_val = torch.FloatTensor(ts.X.tondarray())  # Convert validation data to tensors

nn_model.eval()  # Ensure the model is in evaluation mode
with torch.no_grad():
    logits_train = nn_model(test_tensor_train)
    logits_val = nn_model(test_tensor_val)

# Convert tensor predictions to numpy for metric calculations
logits_train_np = logits_train.numpy()
logits_val_np = logits_val.numpy()

# Apply softmax to convert logits to probabilities
y_prob_train = softmax(logits_train_np)[:, 1]
y_prob_val = softmax(logits_val_np)[:, 1]

# Calculate and print metrics
print("Train ROC AUC Score:", roc_auc_score(tr.Y.tondarray(), y_prob_train))
print("Validation ROC AUC Score:", roc_auc_score(ts.Y.tondarray(), y_prob_val))

Train ROC AUC Score: 0.9640289679369974
Validation ROC AUC Score: 0.9736723672367236


### Attack

La siguiente celda presenta la implementación de un ataque adversario utilizando el método de Descenso de Gradiente Proyectado Mínimo Cuadrado (PGD-LS). El objetivo de este tipo de ataque es explorar cómo se pueden alterar sutilmente los datos de entrada de un modelo de clasificación para inducir errores en las predicciones del modelo.
Configuración del Ataque

- Parámetros del Ataque: Se define un espacio de parámetros que incluye el máximo de perturbación (dmax), la tasa de aprendizaje (eta), y el número máximo de iteraciones (max_iter). Estos parámetros ayudan a regular la intensidad y la duración del ataque.
- Ejecución: Se realiza una búsqueda exhaustiva (grid search) para encontrar la combinación de parámetros que resulte más efectiva en engañar al modelo, probando diversas configuraciones y evaluando su impacto.
- Evaluación de Resultados: Se mide la eficacia del ataque por su capacidad para cambiar la etiqueta predicha de los datos de entrada, y la eficiencia por el número de evaluaciones de gradiente necesarias para lograr este cambio.

In [30]:
import numpy as np
from secml.adv.attacks.evasion import CAttackEvasionPGDLS
from secml.array import CArray

# Define the grid of parameters
dmax_values = [5, 10, 15]  # Higher values
eta_values = [0.01, 0.05, 0.1, 0.2, 0.5, 0.8, 1, 1.5, 3, 5, 10]  # Broader range
max_iter_values = [3000, 4000, 7000]  # More iterations

# Variable to store the best parameters and results
best_params = None
best_success = False
best_evaluations = float('inf')  # Less gradient evaluations might mean more efficient attack

# Loop over all combinations of parameters
for dmax in dmax_values:
    for eta in eta_values:
        for max_iter in max_iter_values:
            pgd_ls_attack = CAttackEvasionPGDLS(
                classifier=clf,
                distance='l1',
                dmax=dmax,
                lb=0, ub=1,
                solver_params={
                    'eta': eta,
                    'eta_min': eta / 3,  # Adjust eta_min to be a fraction of eta
                    'max_iter': max_iter,
                    'eps': 1e-4
                },
                y_target=None,  # Non-targeted attack
                double_init_ds=double_init_ds,
                double_init=True
            )

            # Run the attack
            try:
                _, _, adv_ds_pgdls, _ = pgd_ls_attack.run(x0, y0)
                y_pred_adv = adv_ds_pgdls.Y.item()
                n_evals = pgd_ls_attack.grad_eval
                print(f"\n\nTesting dmax={dmax}, eta={eta}, max_iter={max_iter}:")
                print("Adversarial example label: ", y_pred_adv)
                print("Number of classifier gradient evaluations: ", n_evals)

                # Check if the attack changed the label and if it is more efficient
                if y_pred_adv != y0.item() and n_evals < best_evaluations:
                    best_params = {'dmax': dmax, 'eta': eta, 'max_iter': max_iter}
                    best_evaluations = n_evals
                    best_success = True
                    print("New best attack found!")

            except Exception as e:
                print("Error during attack execution:", e)

# Print the best parameters if a successful attack was found
if best_success:
    print("Best successful attack parameters:")
    print(best_params)
    print(f"Number of evaluations for best attack: {best_evaluations}")
else:
    print("No successful attack was found.")




Testing dmax=5, eta=0.01, max_iter=3000:
Adversarial example label:  0
Number of classifier gradient evaluations:  8


Testing dmax=5, eta=0.01, max_iter=4000:
Adversarial example label:  0
Number of classifier gradient evaluations:  8


Testing dmax=5, eta=0.01, max_iter=7000:
Adversarial example label:  0
Number of classifier gradient evaluations:  8


Testing dmax=5, eta=0.05, max_iter=3000:
Adversarial example label:  0
Number of classifier gradient evaluations:  8


Testing dmax=5, eta=0.05, max_iter=4000:
Adversarial example label:  0
Number of classifier gradient evaluations:  8


Testing dmax=5, eta=0.05, max_iter=7000:
Adversarial example label:  0
Number of classifier gradient evaluations:  8


Testing dmax=5, eta=0.1, max_iter=3000:
Adversarial example label:  0
Number of classifier gradient evaluations:  5


Testing dmax=5, eta=0.1, max_iter=4000:
Adversarial example label:  0
Number of classifier gradient evaluations:  5


Testing dmax=5, eta=0.1, max_iter=7000:
Adversar

La siguiente celda demuestra la implementación de un ataque adversario utilizando el método de Descenso de Gradiente Proyectado (PGD). El objetivo de este enfoque es alterar los datos de entrada a un modelo de clasificación de tal manera que se induzca al error en sus predicciones, alterando las entradas mínimamente pero de manera efectiva.
Configuración del Ataque

- Parámetros del Ataque: Se define un espacio amplio de parámetros que incluye:
  
  1. dmax: Los valores máximos de perturbación que pueden aplicarse a los datos.

  2. eta: La tasa de aprendizaje o tamaño de paso usado para ajustar la entrada en cada iteración del ataque.

  3. max_iter: El número máximo de iteraciones permitidas para cada configuración del ataque, permitiendo así una exploración profunda de posibles perturbaciones.

- Ejecución: Utilizamos una búsqueda exhaustiva para probar combinaciones de estos parámetros, evaluando cómo cada configuración afecta la habilidad del modelo para mantener la precisión de sus clasificaciones originales.

- Evaluación de Resultados: Cada ataque es evaluado no solo por su capacidad para cambiar la clasificación predicha de un ejemplo de prueba, sino también por la eficiencia del ataque, medida en número de evaluaciones de gradiente necesarias para lograr el cambio.

In [39]:
import numpy as np
from secml.adv.attacks.evasion import CAttackEvasionPGD
from secml.array import CArray

# Define the grid of parameters
dmax_values = [5, 10, 15]  # Higher values
eta_values = [0.01, 0.05, 0.1, 0.2, 0.5, 0.8, 1, 1.5, 3, 5, 10]  # Broader range
max_iter_values = [3000, 4000, 7000]  # More iterations

# Variable to store the best parameters and results
best_params = None
best_success = False
best_evaluations = float('inf')  # Less gradient evaluations might mean more efficient attack

# Loop over all combinations of parameters
for dmax in dmax_values:
    for eta in eta_values:
        for max_iter in max_iter_values:
            pgd_ls_attack = CAttackEvasionPGD(
                classifier=clf,
                distance='l1',
                dmax=dmax,
                lb=0, ub=1,
                solver_params={
                    'eta': eta,
                    'max_iter': max_iter,
                    'eps': 1e-4
                },
                y_target=None,  # Non-targeted attack
                double_init_ds=double_init_ds,
                double_init=True
            )

            # Run the attack
            try:
                _, _, adv_ds_pgdls, _ = pgd_ls_attack.run(x0, y0)
                y_pred_adv = adv_ds_pgdls.Y.item()
                n_evals = pgd_ls_attack.grad_eval
                print(f"\n\nTesting dmax={dmax}, eta={eta}, max_iter={max_iter}:")
                print("Adversarial example label: ", y_pred_adv)
                print("Number of classifier gradient evaluations: ", n_evals)

                # Check if the attack changed the label and if it is more efficient
                if y_pred_adv != y0.item() and n_evals < best_evaluations:
                    best_params = {'dmax': dmax, 'eta': eta, 'max_iter': max_iter}
                    best_evaluations = n_evals
                    best_success = True
                    print("New best attack found!")

            except Exception as e:
                print("Error during attack execution:", e)

# Print the best parameters if a successful attack was found
if best_success:
    print("Best successful attack parameters:")
    print(best_params)
    print(f"Number of evaluations for best attack: {best_evaluations}")
else:
    print("No successful attack was found.")

Error during attack execution: x_init CArray([0.       0.       0.       ... 0.235294 0.056191 0.78    ]) is outside of feasible domain.
Error during attack execution: x_init CArray([0.       0.       0.       ... 0.235294 0.056191 0.78    ]) is outside of feasible domain.
Error during attack execution: x_init CArray([0.       0.       0.       ... 0.235294 0.056191 0.78    ]) is outside of feasible domain.


Testing dmax=5, eta=0.05, max_iter=3000:
Adversarial example label:  0
Number of classifier gradient evaluations:  133


Testing dmax=5, eta=0.05, max_iter=4000:
Adversarial example label:  0
Number of classifier gradient evaluations:  133


Testing dmax=5, eta=0.05, max_iter=7000:
Adversarial example label:  0
Number of classifier gradient evaluations:  133


Testing dmax=5, eta=0.1, max_iter=3000:
Adversarial example label:  0
Number of classifier gradient evaluations:  118


Testing dmax=5, eta=0.1, max_iter=4000:
Adversarial example label:  0
Number of classifier gradient eval

Tras ejecutar ambos metodos, no hemos conseguido obtener ningun resultado en el ayaque. Esto, segun informacion que hemos recopilado de diferentes fuentes, puede deberse a varias razones, cada una de las cuales apunta a diferentes aspectos de la resistencia del modelo, la naturaleza del conjunto de datos o los parámetros y restricciones establecidos para el ataque. Vamos a desglosar las posibles razones y los ajustes que podrías considerar para obtener potencialmente un ataque exitoso:

1. Robustez del modelo

    Robustez inherente: Su modelo puede ser intrínsecamente robusto frente a los tipos de perturbaciones que se intentan. Esto puede deberse a la arquitectura del modelo, a las características en las que se basa o a cómo se entrenó (por ejemplo, con regularización, abandono o incluso entrenamiento adversarial).

2. Parámetros de ataque

    dmax (Perturbación máxima): Si dmax es demasiado bajo, el ataque puede no ser capaz de explorar lo suficiente el espacio de entrada para encontrar ejemplos adversarios exitosos.

    eta (Tamaño del paso): Si eta es demasiado pequeño, el ataque podría hacer sólo ajustes mínimos, insuficientes para cruzar los límites de decisión. Por el contrario, si es demasiado grande, podría sobrepasar las perturbaciones adversarias óptimas.

    max_iter (Iteraciones máximas): Si no se proporcionan suficientes iteraciones, es posible que el ataque no tenga suficientes oportunidades para ajustar la entrada de forma eficaz.

3. Función de pérdida e información de gradiente

    Enmascaramiento/blindaje del gradiente: A veces, debido a técnicas específicas de entrenamiento de modelos o a pasos de preprocesamiento de datos, la información de gradiente puede no ser útil o ser engañosa (fenómeno conocido como enmascaramiento de gradiente).

4. Ejecución y límites computacionales

    Limitación de recursos: Los recursos computacionales limitados también pueden restringir la profundidad y amplitud de una búsqueda en cuadrícula.